In [ ]:
import pandas as pd
from matplotlib.patches import Rectangle
import utils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import csv
import random
import os
import json
import math

In [ ]:
#---------------------------------------------------
#
# Config *** TO UPDATE ***
#
#---------------------------------------------------

# *** TO UPDATE: change the team number with your own team id
TEAM_ID = "Example 1"

# Gaze data File
GAZE_DATA_FILE = "../raw-data/"+TEAM_ID+"/gaze data/data.csv"

# File with AOIS definitions 
AOIS_DEFINITION_FILE = "../raw-data/"+TEAM_ID+"/aois definitions/aois.csv"

# Path of the stimulus images for the different tasks 
STIMULUS_IMAGE_WITH_AOIS_PATH = "../raw-data/"+TEAM_ID+"/stimulus image with aois/"

# Numbers of rows with meta-data to skip when creating the dataframe
META_DATA_ROWS_IN_GAZE_DATA_FILE = 25

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 1. Gaze Data import and computation of X and Y gaze coordinates
#
#
#----------------------------------------------------------------------------------------

In [ ]:
#Read gaze data using pandas library
data = pd.read_csv(GAZE_DATA_FILE, skiprows=META_DATA_ROWS_IN_GAZE_DATA_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# preview gaze data
data.head()

In [ ]:
# Get the participants list
print(f'Participants list {data["Respondent"].unique()}')

In [ ]:
# Get the tasks list
print(f'tasks list {data["SourceStimuliName"].unique()}')

In [ ]:
# Replace -1 values with NaN
data = data.replace(-1,np.nan)

# compute the mean X and Y gaze positions based on the left and right eyes
data['GazeX'] = data[['ET_GazeLeftx','ET_GazeRightx']].mean(axis=1)
data['GazeY'] = data[['ET_GazeLefty','ET_GazeRighty']].mean(axis=1)

In [ ]:
# preview gaze data with mean X and Y gaze positions columns
data.head()

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 4. Data exploration and visual quality assessment
# Projecting gaze points on Areas of Interests (AOIs)
# 
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read file with AOI coordinates
""" Dataframe columns
AOI: name of the AOI
p1x: starting point of the AOI rectangle on the x axis
p1y: starting point of the AOI rectangle on the y axis
p2x: ending point of the AOI rectangle on the x axis
p2y: ending point of the AOI rectangle on the y axis
task: name of the task
"""
aois = pd.read_csv(AOIS_DEFINITION_FILE)

# Compute the width and the height of the AOIs based on their start/ending points
aois['width'] = aois['p2x']-aois['p1x']
aois['height'] = aois['p2y']-aois['p1y']

display(aois)

In [ ]:
# Iteratate over the participants and tasks, then plot gazes and AOIs for each participant and task
for participant in data['Respondent'].unique():
    
    for task in data["SourceStimuliName"].unique():
        
        # Path to the stimulus image
        image_path = STIMULUS_IMAGE_WITH_AOIS_PATH+task+'.png'

        # filter in the participant and task data
        participantTaskData= data[(data['Respondent']==participant) & (data['SourceStimuliName']==task)]

        # Initiate a coordinate system with a background image
        fig, ax = utils.initPlotContainer(image_path)

        # set title
        ax.set_title(f'Participant: {participant}, Task: {task}', fontsize=30)

        # Plot gazes
        ax.plot(participantTaskData['GazeX'], participantTaskData['GazeY'],'o',alpha=0.5, markersize=1, color='red')

        #show and close
        plt.show(fig)
        plt.close(fig)

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 4. Assign gaze points to AOIs
#
#
#----------------------------------------------------------------------------------------

In [ ]:
# copy gazeData to gazeDataWithAOIs
gazeDataWithAOIs = data.copy(deep=True)

# iterate over the rows in aois dataframe
for index, row in aois.iterrows():
    # get the name of AOI
    aoiName = row['AOI']
    # add a new column with key=aoiName and value=(0 or 1) depending on whether the gaze point in within the AOI in question 
    gazeDataWithAOIs[aoiName] = data.apply(lambda x : utils.pointInRect(x['SourceStimuliName'],x['GazeX'],x['GazeY'],row['task'],row['p1x'],row['p1y'],row['p2x'],row['p2y']),axis=1)

In [ ]:
# preview gaze data with AOIs mapping
gazeDataWithAOIs.head()

In [ ]:
# export gazeDataWithAOIs as csv
gazeDataWithAOIs.to_csv('data/gazeDataWithAOIs.csv',  index=False)